In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio
from hyperopt import fmin, tpe, hp
from sklearn.metrics import mean_absolute_error

df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [2]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

def MAE(actual, pred):
    return mean_absolute_error(actual, pred)

### Agrego Features

In [3]:
df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

### Filtro columnas

In [26]:
df_XGBoost = df_train_f.copy().drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 'id', 'fecha'], axis=1)

df_XGBoost = df_XGBoost[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'prop_frecuente', 'es_ciudad_centrica', 'porcentaje_metros', 'diferencia_metros',
       'delincuencia','turismo','promedio_precio_ciudad', 'promedio_precio_tipo_propiedad',
       'anio','promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_id_zona', 'promedio_por_mes']]

df_eval = df_train_f.copy().drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 
                                 'id', 'fecha'], axis=1)

df_eval = df_eval[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'prop_frecuente', 'es_ciudad_centrica', 'porcentaje_metros', 'diferencia_metros',
       'delincuencia','turismo','promedio_precio_ciudad', 'promedio_precio_tipo_propiedad',
       'anio','promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_id_zona', 'promedio_por_mes']]


features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'prop_frecuente', 'es_ciudad_centrica', 'porcentaje_metros', 'diferencia_metros',
       'delincuencia','turismo','promedio_precio_ciudad', 'promedio_precio_tipo_propiedad',
       'anio','promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_id_zona', 'promedio_por_mes']


## Busco los mejores hiperparametros

In [28]:
def xgb_eval(args):
    
    # Modelo - Ver hiperparametros
    colsample_bytree, learning_rate, max_depth, alpha, n_estimators, test_size = args
    
    # Preparacion de los datos
    df_XGBoost_train, df_XGBoost_test = train_test_split(df_XGBoost, test_size=test_size, random_state=1)
    x_test = df_XGBoost_test.drop('precio', axis=1)
    y_test = df_XGBoost_test['precio']
    x_train = df_XGBoost_train.drop('precio', axis=1)
    y_train = df_XGBoost_train['precio']
    x_eval = df_eval.drop('precio', axis=1)
    y_eval = df_eval['precio']
    
    xg_train = xgb.DMatrix(x_train, label=y_train)
    xg_test = xgb.DMatrix(x_test, label=y_test)
    dfg_test = xgb.DMatrix(x_eval, label = y_eval)

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    params = {
        'objective':'reg:squarederror', 
        'colsample_bytree' : colsample_bytree , 
        'learning_rate' : learning_rate,
        'max_depth' : max_depth, 
        'alpha' : alpha,
        'n_estimators' : n_estimators,
    }

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 25
    
    xg_reg = xgb.train(params, 
                    xg_train, 
                    num_round, 
                    watchlist
                   )

    y_pred_eval = xg_reg.predict(dfg_test)
    return MAE(y_eval, y_pred_eval)

space = [hp.quniform('colsample_bytree', 0.6, 1, 0.05), hp.uniform('learning_rate', 0.05, 0.35),
         hp.quniform("max_depth", 5, 20, 1),hp.loguniform("alpha", np.log(0.1), np.log(10)),
        hp.quniform("n_estimators", 10, 600, 10), hp.uniform("test_size", 0.01,0.8)]

hps = fmin(xgb_eval, space=space, algo=tpe.suggest, max_evals=50)

display(hps)

  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \



[0]	train-rmse:2.61936e+06	test-rmse:2.56777e+06    

[1]	train-rmse:2.08532e+06	test-rmse:2.05581e+06    

[2]	train-rmse:1.69027e+06	test-rmse:1.68545e+06    

[3]	train-rmse:1.39697e+06	test-rmse:1.41812e+06    

[4]	train-rmse:1.18654e+06	test-rmse:1.23402e+06    

[5]	train-rmse:1.04103e+06	test-rmse:1.11198e+06    

[6]	train-rmse:930938	test-rmse:1.02701e+06         

[7]	train-rmse:851944	test-rmse:971358              

[8]	train-rmse:793188	test-rmse:935502              

[9]	train-rmse:752894	test-rmse:912235              

[10]	train-rmse:721169	test-rmse:897213             

[11]	train-rmse:693712	test-rmse:886586             

[12]	train-rmse:674879	test-rmse:879666             

[13]	train-rmse:659424	test-rmse:875772             

[14]	train-rmse:644632	test-rmse:872453             

[15]	train-rmse:631594	test-rmse:869817             

[16]	train-rmse:618431	test-rmse:868551             

[17]	train-rmse:606053	test-rmse:866435             

[18]	train-rmse:595648	test-

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \



[0]	train-rmse:2.7435e+06	test-rmse:2.66084e+06                            

[1]	train-rmse:2.27586e+06	test-rmse:2.2276e+06                            

[2]	train-rmse:1.89745e+06	test-rmse:1.88588e+06                           

[3]	train-rmse:1.59153e+06	test-rmse:1.6217e+06                            

[4]	train-rmse:1.34394e+06	test-rmse:1.42009e+06                           

[5]	train-rmse:1.14316e+06	test-rmse:1.27163e+06                           

[6]	train-rmse:979993	test-rmse:1.15832e+06                                

[7]	train-rmse:847234	test-rmse:1.07603e+06                                

[8]	train-rmse:739915	test-rmse:1.01039e+06                                

[9]	train-rmse:652352	test-rmse:964754                                     

[10]	train-rmse:580731	test-rmse:934650                                    

[11]	train-rmse:522428	test-rmse:911077                                    

[12]	train-rmse:474450	test-rmse:897342                                    


...

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \



[0]	train-rmse:2.70681e+06	test-rmse:2.62186e+06                              

[1]	train-rmse:2.21796e+06	test-rmse:2.16219e+06                              

[2]	train-rmse:1.83295e+06	test-rmse:1.81712e+06                              

[3]	train-rmse:1.53395e+06	test-rmse:1.56921e+06                              

[4]	train-rmse:1.30474e+06	test-rmse:1.37965e+06                              

[5]	train-rmse:1.12636e+06	test-rmse:1.23913e+06                              

[6]	train-rmse:970728	test-rmse:1.12748e+06                                   

[7]	train-rmse:846261	test-rmse:1.04643e+06                                   

[8]	train-rmse:746991	test-rmse:993443                                        

[9]	train-rmse:673016	test-rmse:953992                                        

[10]	train-rmse:612074	test-rmse:927566                                       

[11]	train-rmse:562402	test-rmse:912077                                       

[12]	train-rmse:520908	test-rmse:900026 

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \



[0]	train-rmse:2.94349e+06	test-rmse:2.94222e+06                              

[1]	train-rmse:2.60528e+06	test-rmse:2.6197e+06                               

[2]	train-rmse:2.31156e+06	test-rmse:2.3453e+06                               

[3]	train-rmse:2.05375e+06	test-rmse:2.11062e+06                              

[4]	train-rmse:1.83221e+06	test-rmse:1.91364e+06                              

[5]	train-rmse:1.64349e+06	test-rmse:1.75221e+06                              

[6]	train-rmse:1.47064e+06	test-rmse:1.6076e+06                               

[7]	train-rmse:1.31794e+06	test-rmse:1.48711e+06                              

[8]	train-rmse:1.1826e+06	test-rmse:1.38723e+06                               

[9]	train-rmse:1.06653e+06	test-rmse:1.30454e+06                              

[10]	train-rmse:962489	test-rmse:1.23546e+06                                  

[11]	train-rmse:870530	test-rmse:1.17797e+06                                  

[12]	train-rmse:789185	test-rmse:1.13082

{'alpha': 0.44213195307516456,
 'colsample_bytree': 0.75,
 'learning_rate': 0.26130498561827115,
 'max_depth': 18.0,
 'n_estimators': 420.0,
 'test_size': 0.11065394016279906}

### Busco los mejores features

In [37]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            # 'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['tipo_propiedad_compartida',
                 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
                 'promedio_metros_totales_provincia',
                 'promedio_metros_cubiertos_provincia', 'porcentaje_metros',
                 'diferencia_metros', 'delincuencia', 'turismo',
                 'promedio_precio_ciudad', 'promedio_id_zona',
                 'promedio_precio_tipo_propiedad','es_antigua',
                 'promedio_por_mes', 'promedio_precio_habitaciones',
                 'promedio_precio_habitaciones_banos_garages','cantidad_inquilinos',
                 'promedio_precio_banos_garages', 'promedio_precio_booleanos',
                 'metros_totales_normalizados', 'metros_cubiertos_normalizados','escomercial',
                 'promedio_metros_cub_tipo_propiedad', 'anio', 'mes', 'dia','trimestre']


In [51]:
hps = {'alpha': 0.5686199348949729,
 'colsample_bytree': 0.9,
 'learning_rate': 0.174921124703759,
 'max_depth': 10.0,
 'n_estimators': 70.0,
}

alpha = hps['alpha']
colsample_bytree = hps['colsample_bytree']
max_depth = hps['max_depth']
learning_rate = hps['learning_rate']
n_estimators = hps['n_estimators']

n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

params = {
        'objective':'reg:squarederror', 
        'colsample_bytree' : colsample_bytree , 
        'learning_rate' : learning_rate,
        'max_depth' : max_depth, 
        'alpha' : alpha,
        'n_estimators' : n_estimators,
}


base_train = 0
base_test = 0
base_eval = 0
for i in ['None'] + features_test:
    x_train, x_test, y_train, y_test = dividir_dataset(df_train_f, 'precio', features + [i])

    xg_train = xgb.DMatrix(x_train, label=y_train)
    xg_test = xgb.DMatrix(x_test, label=y_test)
    xg_train = xgb.DMatrix(x_train, label=y_train)
    xg_test = xgb.DMatrix(x_test, label=y_test)

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 50
    
    xg_reg = xgb.train(params, 
                    xg_train, 
                    num_round, 
                    watchlist
                   )
    
    x_train_predict = xgb.DMatrix(x_train, label=y_train)
    x_test_predict = xgb.DMatrix(x_test, label=y_test)
    x_eval_predict = xgb.DMatrix(filtrar_features(df_test_f, features + [i]))
    
    y_pred_test = xg_reg.predict(x_test_predict)
    y_pred_train = xg_reg.predict(x_train_predict)
    y_pred_eval = xg_reg.predict(x_eval_predict)

    xgb_mae_train = MAE(y_train, y_pred_train)
    xgb_mae = MAE(y_test, y_pred_test)
    xgb_mae_eval = MAE(df_test_f['precio'].values, y_pred_eval)

    print(f"MAE LightGBM (train): {xgb_mae_train:.5f}")
    print(f"MAE LightGBM (test): {xgb_mae:.5f}")
    print(f"MAE LightGBM (eval): {xgb_mae_eval:.5f}")
    if i is not 'None':
        print(f"Overfitting (base_eval - base_test) - (eval - test) - {i}: {(base_eval - base_test) - (xgb_mae_eval - xgb_mae)}")
        print(f"Diff evaluation (base_eval - eval)                  - {i}: {base_eval - xgb_mae_eval}")
        print(f"Diff train (base_train - train)                     - {i}: {base_train - xgb_mae_train}")
    else:
        base_train = xgb_mae_train
        base_test = xgb_mae
        base_eval = xgb_mae_eval


/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-rmse:2.87824e+06	test-rmse:2.87786e+06
[1]	train-rmse:2.51968e+06	test-rmse:2.52366e+06
[2]	train-rmse:2.24998e+06	test-rmse:2.25753e+06
[3]	train-rmse:2.04202e+06	test-rmse:2.05432e+06
[4]	train-rmse:1.87207e+06	test-rmse:1.89218e+06
[5]	train-rmse:1.74259e+06	test-rmse:1.771e+06
[6]	train-rmse:1.64786e+06	test-rmse:1.68267e+06
[7]	train-rmse:1.57894e+06	test-rmse:1.62025e+06
[8]	train-rmse:1.5241e+06	test-rmse:1.57309e+06
[9]	train-rmse:1.485e+06	test-rmse:1.54103e+06
[10]	train-rmse:1.45772e+06	test-rmse:1.51905e+06
[11]	train-rmse:1.43426e+06	test-rmse:1.50157e+06
[12]	train-rmse:1.41738e+06	test-rmse:1.49004e+06
[13]	train-rmse:1.40284e+06	test-rmse:1.48104e+06
[14]	train-rmse:1.3911e+06	test-rmse:1.47557e+06
[15]	train-rmse:1.38097e+06	test-rmse:1.46961e+06
[16]	train-rmse:1.3723e+06	test-rmse:1.46544e+06
[17]	train-rmse:1.36391e+06	test-rmse:1.46292e+06
[18]	train-rmse:1.35761e+06	test-rmse:1.46036e+06
[19]	train-rmse:1.35168e+06	test-rmse:1.4585e+06
[20]	train-rmse:1.

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-rmse:2.87486e+06	test-rmse:2.8743e+06
[1]	train-rmse:2.51538e+06	test-rmse:2.51956e+06
[2]	train-rmse:2.24095e+06	test-rmse:2.24942e+06
[3]	train-rmse:2.03072e+06	test-rmse:2.04429e+06
[4]	train-rmse:1.86295e+06	test-rmse:1.884e+06
[5]	train-rmse:1.73666e+06	test-rmse:1.76412e+06
[6]	train-rmse:1.64038e+06	test-rmse:1.67496e+06
[7]	train-rmse:1.57076e+06	test-rmse:1.61174e+06
[8]	train-rmse:1.5192e+06	test-rmse:1.56583e+06
[9]	train-rmse:1.48129e+06	test-rmse:1.53402e+06
[10]	train-rmse:1.45309e+06	test-rmse:1.51145e+06
[11]	train-rmse:1.43071e+06	test-rmse:1.4945e+06
[12]	train-rmse:1.41460e+06	test-rmse:1.48277e+06
[13]	train-rmse:1.4005e+06	test-rmse:1.47348e+06
[14]	train-rmse:1.39013e+06	test-rmse:1.46768e+06
[15]	train-rmse:1.38109e+06	test-rmse:1.46316e+06
[16]	train-rmse:1.37181e+06	test-rmse:1.45862e+06
[17]	train-rmse:1.36416e+06	test-rmse:1.45529e+06
[18]	train-rmse:1.35753e+06	test-rmse:1.45277e+06
[19]	train-rmse:1.35193e+06	test-rmse:1.45093e+06
[20]	train-rmse:

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


...

[0]	train-rmse:2.87537e+06	test-rmse:2.87557e+06
[1]	train-rmse:2.51608e+06	test-rmse:2.52102e+06
[2]	train-rmse:2.24211e+06	test-rmse:2.25351e+06
[3]	train-rmse:2.03149e+06	test-rmse:2.04858e+06
[4]	train-rmse:1.86456e+06	test-rmse:1.88981e+06
[5]	train-rmse:1.73796e+06	test-rmse:1.77072e+06
[6]	train-rmse:1.64233e+06	test-rmse:1.68296e+06
[7]	train-rmse:1.57152e+06	test-rmse:1.61947e+06
[8]	train-rmse:1.51926e+06	test-rmse:1.57451e+06
[9]	train-rmse:1.48061e+06	test-rmse:1.54235e+06
[10]	train-rmse:1.45176e+06	test-rmse:1.51984e+06
[11]	train-rmse:1.42792e+06	test-rmse:1.50251e+06
[12]	train-rmse:1.41091e+06	test-rmse:1.49169e+06
[13]	train-rmse:1.3971e+06	test-rmse:1.48241e+06
[14]	train-rmse:1.38427e+06	test-rmse:1.47519e+06
[15]	train-rmse:1.37469e+06	test-rmse:1.47045e+06
[16]	train-rmse:1.36642e+06	test-rmse:1.46609e+06
[17]	train-rmse:1.35746e+06	test-rmse:1.46314e+06
[18]	train-rmse:1.35127e+06	test-rmse:1.46119e+06
[19]	train-rmse:1.34435e+06	test-rmse:1.45946e+06
[20]	train-

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-rmse:2.87557e+06	test-rmse:2.87572e+06
[1]	train-rmse:2.51633e+06	test-rmse:2.52114e+06
[2]	train-rmse:2.24243e+06	test-rmse:2.25316e+06
[3]	train-rmse:2.03213e+06	test-rmse:2.04834e+06
[4]	train-rmse:1.86485e+06	test-rmse:1.88876e+06
[5]	train-rmse:1.73803e+06	test-rmse:1.77002e+06
[6]	train-rmse:1.64242e+06	test-rmse:1.68258e+06
[7]	train-rmse:1.57198e+06	test-rmse:1.61952e+06
[8]	train-rmse:1.51982e+06	test-rmse:1.57475e+06
[9]	train-rmse:1.48163e+06	test-rmse:1.54333e+06
[10]	train-rmse:1.45275e+06	test-rmse:1.52051e+06
[11]	train-rmse:1.42957e+06	test-rmse:1.50374e+06
[12]	train-rmse:1.41285e+06	test-rmse:1.49277e+06
[13]	train-rmse:1.39889e+06	test-rmse:1.4844e+06
[14]	train-rmse:1.38736e+06	test-rmse:1.47831e+06
[15]	train-rmse:1.37633e+06	test-rmse:1.47328e+06
[16]	train-rmse:1.36794e+06	test-rmse:1.46909e+06
[17]	train-rmse:1.35995e+06	test-rmse:1.46608e+06
[18]	train-rmse:1.35292e+06	test-rmse:1.46352e+06
[19]	train-rmse:1.3461e+06	test-rmse:1.46198e+06
[20]	train-r

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[0]	train-rmse:2.87573e+06	test-rmse:2.8756e+06
[1]	train-rmse:2.51656e+06	test-rmse:2.5211e+06
[2]	train-rmse:2.24326e+06	test-rmse:2.25299e+06
[3]	train-rmse:2.0333e+06	test-rmse:2.04845e+06
[4]	train-rmse:1.86665e+06	test-rmse:1.88966e+06
[5]	train-rmse:1.74049e+06	test-rmse:1.77073e+06
[6]	train-rmse:1.64531e+06	test-rmse:1.68301e+06
[7]	train-rmse:1.57604e+06	test-rmse:1.61989e+06
[8]	train-rmse:1.5234e+06	test-rmse:1.57448e+06
[9]	train-rmse:1.48539e+06	test-rmse:1.54338e+06
[10]	train-rmse:1.4571e+06	test-rmse:1.52106e+06
[11]	train-rmse:1.43443e+06	test-rmse:1.50387e+06
[12]	train-rmse:1.41792e+06	test-rmse:1.49243e+06
[13]	train-rmse:1.40442e+06	test-rmse:1.48363e+06
[14]	train-rmse:1.39274e+06	test-rmse:1.47748e+06
[15]	train-rmse:1.38326e+06	test-rmse:1.47316e+06
[16]	train-rmse:1.37561e+06	test-rmse:1.46934e+06
[17]	train-rmse:1.367e+06	test-rmse:1.46508e+06
[18]	train-rmse:1.3617e+06	test-rmse:1.4624e+06
[19]	train-rmse:1.35372e+06	test-rmse:1.45942e+06
[20]	train-rmse:1.3

## Predicción

In [32]:
# Preparacion de los datos
df_XGBoost_train, df_XGBoost_test = train_test_split(df_XGBoost, test_size=0.11065394016279906, random_state=1)
x_test = df_XGBoost_test.drop('precio', axis=1)
y_test = df_XGBoost_test['precio']
x_train = df_XGBoost_train.drop('precio', axis=1)
y_train = df_XGBoost_train['precio']
x_eval = df_eval.drop('precio', axis=1)
y_eval = df_eval['precio']

# Modelo - Ver hiperparametros
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                          colsample_bytree = 0.75, 
                          learning_rate =0.26130498561827115,
                          max_depth = 18, 
                          alpha = 0.44213195307516456, 
                          n_estimators = 420,
                         )

# Entrenamiento
xg_reg.fit(x_train, y_train)

# Prediccion
y_pred_test = xg_reg.predict(x_test)
y_pred_train = xg_reg.predict(x_train)


linear_rmsle_train = RMSLE(y_train, y_pred_train)
linear_rmsle = RMSLE(y_test, y_pred_test)
linear_mae_train = MAE(y_train, y_pred_train)
linear_mae = MAE(y_test, y_pred_test)

y_pred_eval = xg_reg.predict(x_eval)
linear_mae_eval = MAE(y_eval, y_pred_eval)


print(f"RMSLE LinearRegression (train): {linear_rmsle_train:.5f}")
print(f"RMSLE LinearRegression: {linear_rmsle:.5f}")
print(f"MAE (train): {linear_mae_train:.5f}")
print(f"MAE: {linear_mae:.5f}")
print(f"MAE (eval): {linear_mae_eval:.5f}")


/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSLE LinearRegression (train): 0.00746
RMSLE LinearRegression: 0.30494
MAE (train): 1288.17319
MAE: 497024.40142
MAE (eval): 56143.45241


## Predicción final

In [33]:
df_train_final = pd.read_csv('./data/train.csv')
df_test_final = pd.read_csv('./data/test.csv')

df_train_final_f = features_independientes_precio(df_train_final)
df_train_final_f = features_dependientes_precio(df_train_final_f, df_train_final)
df_test_final_f = features_independientes_precio(df_test_final)
df_test_final_f = features_dependientes_precio(df_test_final_f, df_train_final)

In [ ]:
# Filtro columnas
df_XGBoost_fin = df_train_final_f.copy().drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 
                                 'fecha', 'id'], axis=1)

df_XGBoost_fin = df_XGBoost_fin[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'prop_frecuente', 'es_ciudad_centrica', 'porcentaje_metros', 'diferencia_metros',
       'delincuencia','turismo','promedio_precio_ciudad', 'promedio_precio_tipo_propiedad',
       'anio','promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_id_zona', 'promedio_por_mes']]


df_eval_fin = df_test_final_f.copy()
df_eval_fin = df_eval_fin[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos',
       'prop_frecuente', 'es_ciudad_centrica', 'porcentaje_metros', 'diferencia_metros',
       'delincuencia','turismo','promedio_precio_ciudad', 'promedio_precio_tipo_propiedad',
       'anio','promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_id_zona', 'promedio_por_mes']]


# Separo en validacion y en test
f_df_XGBoost_train, f_df_XGBoost_test = train_test_split(df_XGBoost_fin, test_size= 0.11065394016279906, random_state=1)

# Preparacion de los datos
x_test_f = f_df_XGBoost_test.drop('precio', axis=1)
y_test_f = f_df_XGBoost_test['precio']
x_train_f = f_df_XGBoost_train.drop('precio', axis=1)
y_train_f = f_df_XGBoost_train['precio']


# Modelo - Ver hiperparametros
xg_reg_f = xgb.XGBRegressor(objective ='reg:squarederror', 
                          colsample_bytree = 0.75, 
                          learning_rate =0.26130498561827115,
                          max_depth = 18, 
                          alpha = 0.44213195307516456, 
                          n_estimators = 420,
                         )

# Entrenamiento
xg_reg_f.fit(x_train_f, y_train_f)

# Prediccion
y_pred_test_f = xg_reg_f.predict(x_test_f)
y_pred_train_f = xg_reg_f.predict(x_train_f)


linear_rmsle_train_f = RMSLE(y_train_f, y_pred_train_f)
linear_rmsle_f = RMSLE(y_test_f, y_pred_test_f)
linear_mae_train_f = MAE(y_train_f, y_pred_train_f)
linear_mae_f = MAE(y_test_f, y_pred_test_f)


print(f"RMSLE LinearRegression (train): {linear_rmsle_train_f:.5f}")
print(f"RMSLE LinearRegression: {linear_rmsle_f:.5f}")
print(f"MAE (train): {linear_mae_train_f:.5f}")
print(f"MAE: {linear_mae_f:.5f}")


prediccion_final = xg_reg_f.predict(df_eval_fin)
df_test_final_f['target'] = prediccion_final
df_test_final_f[['id', 'target']].to_csv('respuesta.csv', index = False)

/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camix/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


### Verificación de Nulos

In [ ]:
df_test_final_f['target'].isnull().any()

In [ ]:
df_test_final_f['target'].isnull().value_counts()